random forest model will be used to predict whether a customer will be satisfied with their flight experience.

# Imports 

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn. metrics import f1_score, precision_score, recall_score, accuracy_score

<ipython-input-1-23639d984aa5>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
air_data = pd.read_csv("airline.csv")

# EDA 

In [3]:
air_data.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,...,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,...,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,...,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,...,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,...,2,2,2,4,5,3,4,2,30,26.0


In [4]:
air_data.dtypes

satisfaction                          object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
dtype: obj

In [5]:
air_data.shape

(129880, 22)

In [6]:
air_data.isna().any(axis=1).sum()

393

In [7]:
air_data_subset = air_data.dropna(axis=0)

In [8]:
air_data_subset.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,...,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,...,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,...,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,...,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,...,2,2,2,4,5,3,4,2,30,26.0


In [9]:
air_data_subset.isna().sum()

satisfaction                         0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Seat comfort                         0
Departure/Arrival time convenient    0
Food and drink                       0
Gate location                        0
Inflight wifi service                0
Inflight entertainment               0
Online support                       0
Ease of Online booking               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Cleanliness                          0
Online boarding                      0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
dtype: int64

In [10]:
air_data_subset_dummies = pd.get_dummies(air_data_subset, columns = ["Customer Type", "Type of Travel", "Class"])

In [11]:
air_data_subset_dummies

,satisfaction,Age,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,...,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
0,satisfied,65,265,0,0,0,2,2,4,2,...,2,0,0.0,True,False,False,True,False,True,False
1,satisfied,47,2464,0,0,0,3,0,2,2,...,2,310,305.0,True,False,False,True,True,False,False
2,satisfied,15,2138,0,0,0,3,2,0,2,...,2,0,0.0,True,False,False,True,False,True,False
3,satisfied,60,623,0,0,0,3,3,4,3,...,3,0,0.0,True,False,False,True,False,True,False
4,satisfied,70,354,0,0,0,3,4,3,4,...,5,0,0.0,True,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129875,satisfied,29,1731,5,5,5,3,2,5,2,...,2,0,0.0,False,True,False,True,False,True,False
129876,dissatisfied,63,2087,2,3,2,4,2,1,1,...,1,174,172.0,False,True,False,True,True,False,False
129877,dissatisfied,69,2320,3,0,3,3,3,2,2,...,2,155,163.0,False,True,False,True,False,True,False
129878,dissatisfied,66,2450,3,2,3,2,3,2,2,...,2,193,205.0,False,True,False,True,False,True,False


In [12]:
air_data_subset_dummies.dtypes

satisfaction                          object
Age                                    int64
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
Customer Type_Loyal Customer            bool
Customer Type_disloyal Customer         bool
Type of Travel_Business travel          bool
Type of Tr

# Model Building 

In [13]:
y = air_data_subset_dummies["satisfaction"]
x = air_data_subset_dummies.drop("satisfaction", axis=1)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=0)

In [15]:
cv_params = {"n_estimators" : [50, 100],
            "max_depth" : [10, 15],
            "min_samples_leaf" : [0.5, 1],
            "min_samples_split" : [0.001, 0.01],
            "max_features" : ["sqrt"],
            "max_samples" : [.5, .9]}

In [16]:
split_index = [0 if x in x_val.index else -1 for x in x_train.index]
custom_split = PredefinedSplit(split_index)

In [17]:
rf = RandomForestClassifier(random_state=0)

In [18]:
rf_val = GridSearchCV(rf, cv_params, cv = custom_split, refit = "f1", n_jobs = -1, verbose = 1)

In [20]:
%%time

rf_val.fit(x_train, y_train)

Fitting 1 folds for each of 32 candidates, totalling 32 fits
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1min 3s


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ..., -1, -1])),
             estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [10, 15], 'max_features': ['sqrt'],
                         'max_samples': [0.5, 0.9],
                         'min_samples_leaf': [0.5, 1],
                         'min_samples_split': [0.001, 0.01],
                         'n_estimators': [50, 100]},
             refit='f1', verbose=1)

In [21]:
rf_val.best_params_

{'max_depth': 15,
 'max_features': 'sqrt',
 'max_samples': 0.9,
 'min_samples_leaf': 1,
 'min_samples_split': 0.001,
 'n_estimators': 100}

# Results and Evaluation

In [22]:
rf_opt = RandomForestClassifier(n_estimators = 50, max_depth = 50, min_samples_leaf = 1, min_samples_split = 0.001, 
                               max_features = "sqrt", max_samples = 0.9, random_state = 0)

In [23]:
rf_opt.fit(x_train, y_train)

RandomForestClassifier(max_depth=50, max_samples=0.9, min_samples_split=0.001,
                       n_estimators=50, random_state=0)

In [24]:
y_pred = rf_opt.predict(x_test)

In [25]:
pc_test = precision_score(y_test, y_pred, pos_label = "satisfied")
print("the precision score is {pc:.3f}".format(pc = pc_test))

the precision score is 0.950


In [26]:
rc_test = recall_score(y_test, y_pred, pos_label = "satisfied")
print("the recall score is {rc:.3f}".format(rc = rc_test))

the recall score is 0.945


In [27]:
ac_test = accuracy_score(y_test, y_pred)
print("the accuracy score is {ac:.3f}".format(ac = ac_test))

the accuracy score is 0.942


In [28]:
f1_test = f1_score(y_test, y_pred, pos_label = "satisfied")
print("the f1 score is {f1:.3f}".format(f1 = f1_test))

the f1 score is 0.947


In [29]:
print("\nthe precision score is: {pc:.3f}".format(pc = pc_test)),
"for the test set,", "\n which means of all positive predicitions,", "{pc_pct:1f}% predictions are true positive.".format(pc_pct= pc_test * 100)


the precision score is: 0.950


('for the test set,',
 '\n which means of all positive predicitions,',
 '95.012766% predictions are true positive.')

In [30]:
print("\nthe recall score is: {rc:.3f}".format(rc = rc_test),
     "for the test set,",
     "\nwhich means of all cases in test set,",
     "{rc_pct:.1f}% are predected of all real positive cases in test set".format(rc_pct = rc_test * 100))


the recall score is: 0.945 for the test set, 
which means of all cases in test set, 94.5% are predected of all real positive cases in test set


In [31]:
print("\nthe accuracy score is: {ac:.3f}".format(ac = ac_test),
     "for the test set,",
     "\nwhich means of all cases in test set,",
     "{ac_pct:.1f}% are predetcted true positives or true negatives".format(ac_pct = ac_test * 100))


the accuracy score is: 0.942 for the test set, 
which means of all cases in test set, 94.2% are predetcted true positives or true negatives


In [32]:
print("\nthe fi score is: {f1:.3f}".format(f1 = f1_test),
     "for the test set,",
     "\nwhich means the test set's harmonic mean is {f1_pct:.1f}%".format(f1_pct = f1_test * 100))


the fi score is: 0.947 for the test set, 
which means the test set's harmonic mean is 94.7%


In [33]:
table = pd.DataFrame({"Model" : ["Tuned decision tree", "tunned random forest"],
                     "f1" : [0.945422, f1_test],
                     "recall" : [0.935863, rc_test],
                     "precision" : [0.955197, pc_test],
                     "acuracy" : [0.940864, ac_test]
                     }
                    )
table

,Model,f1,recall,precision,acuracy
0,Tuned decision tree,0.945422,0.935863,0.955197,0.940864
1,tunned random forest,0.947306,0.944501,0.950128,0.942450
